In [ ]:
# Change runtime type to GPU for acceleration

!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -U -r yolov5/requirements.txt  # install dependencies
%cd /content/yolov5

import torch
from IPython.display import Image, clear_output  # for displaying images
from utils.google_utils import gdrive_download  # for downloading models/datasets

clear_output()

In [ ]:
%cd /content

# Download my augmented dataset (Dataset: train:306, valid:20, test:14)
!curl -L "https://app.roboflow.com/ds/fYOdAuLmUO?key=CvwxAIKv8R" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
# Check classes 
%cat data.yaml

In [ ]:
# Define number of classes based on YAML for custom model
import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
# Model configuration 
%cat /content/yolov5/models/yolov5s.yaml

In [ ]:
# Enable modification of num_classes
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
# Get yolov5s and change num_classes
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
# train custom yolov5s for total 340 images, 32 batches, 500 epochs, time taken will be displayed
%%time
%cd /content/yolov5/
!python train.py --img 340 --batch 32 --epochs 500 --data '../data.yaml' --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache

In [ ]:
# Trained weights has been saved in path
%ls runs/train/yolov5s_results2/weights

best.pt  last.pt


In [ ]:
# Testing : 
%cd /content/yolov5/

# Bound >= 55% Confidence
!python detect.py --weights runs/train/yolov5s_results2/weights/best.pt --conf 0.55 --source ../test/images
#!python detect.py --weights weights/best.pt --conf 0.58 --source ../test/images

In [ ]:
# Display inference on ALL test images

import glob
from IPython.display import Image, display

# Change exp* value depending of number of trainings
for imageName in glob.glob('/content/yolov5/runs/detect/exp3/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")